# Mobilenet-v2 (width_multiplier=0.5), Pytorch for Sign Language MNIST

That's a quick notebook for training very efficient and accurate network suitable for mobile devices. It achieves 97% top-1 accuracy.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
! nvidia-smi

In [ ]:
#import necessary libraries
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import datasets,transforms,models
import torch.nn.functional as F
from torch import nn,optim

from torch.utils.data import Dataset, TensorDataset
from torch.optim import lr_scheduler

import torchvision
import seaborn as sns

In [ ]:
train_dir='/kaggle/input/sign-language-mnist/sign_mnist_train.csv'
test_dir='/kaggle/input/sign-language-mnist/sign_mnist_test.csv'

train=pd.read_csv(train_dir)
test=pd.read_csv(test_dir)

In [ ]:
#check train data
train.head(10) 

In [ ]:
#check test data
test.head(10)

In [ ]:
#Number of classes we have 

print(train['label'].unique())

print("Number of classes : ",len(train['label'].unique()))

In [ ]:
#obtain all rows and all columns except the 0 index column

train_data = train.iloc[:, 1:].values
print("Number of train images:", train_data.shape[0])
train_labels=train.loc[:, 'label']
print("Number of pixels in each image:", train_data.shape[1])

test_data = test.iloc[:, 1:].values
print("Number of test images:", test_data.shape[0])
test_labels=test.loc[:, 'label']
print("Number of pixels in each image:", test_data.shape[1])



new_train_labels=np.where(train_labels>8, train_labels-1, train_labels)
new_test_labels=np.where(test_labels>8, test_labels-1, test_labels)



unique_val = np.array(new_test_labels)
#np.append (unique_val, 9)
print(np.unique(unique_val))


In [ ]:
train_data.shape , new_train_labels.shape

In [ ]:
from PIL import Image 

Image.open("/kaggle/input/sign-language-mnist/amer_sign2.png")

In [ ]:
letters={0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'k',10:'L',11:'M',12:'N',13:'O',14:'P',15:'Q',16:'R',17:'S',18:'T',19:'U',20:'V',21:'W',22:'X',23:'Y'}

In [ ]:

plt.figure(figsize=(24,8))

for i in range(27):
  
  plt.subplot(3,9,i+1)
  plt.imshow(train_data[i].reshape(28,28))
  plt.axis('off')
  plt.title(letters[int((new_train_labels[i]))])

In [ ]:
plt.figure(figsize = (30,10))
sns.countplot(new_train_labels)

In [ ]:

#Data Augmentation

data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        
        #transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(10,translate=(0.1,0.1)),
        transforms.RandomResizedCrop(size=28,scale=(0.9,1.1), ratio=(0.95, 1.05)),
        transforms.ToTensor(),
]),
    
    'valid': transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
    ]),
}

In [ ]:
class DtProcessing(Dataset):
    
    #initialise the class variables - transform, data, target
    def __init__(self, data, target, transform=None): 
        self.transform = transform
        self.data = data.reshape((-1,28,28)).astype(np.float32)[:,:,:,None]
        # converting target to torch.LongTensor dtype
        self.target = torch.from_numpy(target).long() 
    
    #retrieve the X and y index value and return it
    def __getitem__(self, index): 
        return self.transform(self.data[index]), self.target[index]
    
    #returns the length of the data
    def __len__(self): 
        return len(list(self.data))
      


In [ ]:
#divide train set into train and validation set

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train_data, new_train_labels, test_size = .15, random_state = 42)

In [ ]:
      
dset_train = DtProcessing(X_train, y_train, transform=data_transforms['train'])

train_loader = torch.utils.data.DataLoader(dset_train, batch_size=120,
                                          shuffle=True, num_workers=0)

dset_valid = DtProcessing(X_valid, y_valid, transform=data_transforms['valid'])

valid_loader = torch.utils.data.DataLoader(dset_valid, batch_size=40,
                                          shuffle=True, num_workers=0)


dset_test = DtProcessing(test_data, new_test_labels, transform=data_transforms['valid'])

test_loader =torch.utils.data.DataLoader(dset_test, batch_size=40, shuffle=True)

In [ ]:
import torch.nn as nn
import math


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU(inplace=True)
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU(inplace=True)
    )


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup

        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU(inplace=True),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )

    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000, width_mult=1., gray=False):
        super(MobileNetV2, self).__init__()
        # setting of inverted residual blocks
        self.cfgs = [
            # t, c, n, s
            [1,  16, 1, 1],
            [6,  24, 2, 2],
            [6,  32, 3, 2],
            [6,  64, 4, 2],
            [6,  96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        # building first layer
        input_channel = _make_divisible(32 * width_mult, 4 if width_mult == 0.1 else 8)
        layers = [conv_3x3_bn(3 if not gray else 1, input_channel, 2)]
        # building inverted residual blocks
        block = InvertedResidual
        for t, c, n, s in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 4 if width_mult == 0.1 else 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1280 * width_mult, 4 if width_mult == 0.1 else 8) if width_mult > 1.0 else 1280
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

def mobilenetv2(**kwargs):
    """
    Constructs a MobileNet V2 model
    """
    return MobileNetV2(**kwargs)

In [ ]:
net = mobilenetv2(width_mult=0.5, num_classes=24, gray=True)


In [ ]:
import torch
import os
import glob
import numpy as np
from torch.utils.data import ConcatDataset, DataLoader, random_split

from tqdm import tqdm

STEPS = 100
EPOCHS = 41
DEVICE = "cuda"
np.random.seed(seed=1987)
torch.manual_seed(1987)

def writeToFile(text):
    file_name = "output_full.log"
    with open(file_name, "a+") as f:
        f.write(text + "\n")


def accuracy(output, target, topk=(3,1)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

def validation(model, valid_loader, device, lossf, scoref):
    model.eval()
    loss, score3, score1 = 0, 0, 0
    vlen = len(valid_loader)
    for x, y in valid_loader:
        x = x.squeeze()
        if len(x.shape) != 3:
            # Skip single or empty data
            continue
        x = x.unsqueeze(1)
        y = y.squeeze()
        x, y = x.to(device), y.to(device)
        output = model(x)
        loss += lossf(output, y).item()
        score3 += scoref(output, y)[0].item()
        score1 += scoref(output, y)[1].item()
    model.train()
    return loss / vlen, score3 / vlen, score1 / vlen

In [ ]:

optimizer = torch.optim.Adam(net.parameters(), lr=0.002)
criterion = torch.nn.CrossEntropyLoss()

itr = 1
net.train().cuda()
tloss, score3, score1 = 0, 0, 0
for epoch in range(EPOCHS):
    for x, y in tqdm(train_loader):
        x = x.squeeze()
        if len(x.shape) != 3:
            # Skip single or empty data
            continue
        x = x.unsqueeze(1)
        y = y.squeeze()
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        output = net(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        tloss += loss.item()
        score3 += accuracy(output, y)[0].item()
        score1 += accuracy(output, y)[1].item()
        

        if itr % STEPS == 0:
            writeToFile('Epoch {} Iteration {} -> Train Loss: {:.4f}, MAP@3: {:.3f}, MAP@1: {:.3f} '.format(epoch, itr, tloss / itr,
                                                                                      score3 / itr, score1 / itr))
            tloss, score = 0, 0
        itr += 1
    vloss, vscore3, vscore1 = validation(net, valid_loader, DEVICE, criterion, accuracy)
    writeToFile('Epoch {} -> Valid Loss: {:.4f}, MAP@3: {:.3f}, MAP@1: {:.3f}'.format(epoch, vloss, vscore3, vscore1))
    filename_pth = 'new_checkpoint' + str(epoch) + '_mobilenet.pth'
    torch.save(net.state_dict(), filename_pth)

In [ ]:
net.load_state_dict(torch.load('new_checkpoint36_mobilenet.pth'))

classes=['A','B','C','D','E','F','G','H','I','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y']

In [ ]:


  model = net
  test_loss = 0.0
  class_correct = list(0. for i in range(24))
  class_total = list(0. for i in range(24))

  model.eval()
# iterate over test data
  for data, target in test_loader:

    batch_size = data.size(0)
    #print(batch_size)
    # move tensors to GPU if CUDA is available
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)

    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
  test_loss = test_loss/len(test_loader)
  print('Test Loss: {:.6f}\n'.format(test_loss))

  for i in range(24):
    if class_total[i] > 0 :
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

  print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

In [ ]:
torch.onnx.export(model, data[0][None,...], 'signlang_mobilenet05.onnx')